# Ames Housing Dataset - XGboost Regressor

> Gianmaria Pizzo - 872966@stud.unive.it

These notebooks represent the project submission for the course [Data and Web Mining](https://www.unive.it/data/course/337525) by Professor [Claudio Lucchese](https://www.unive.it/data/people/5590426) at [Ca' Foscari University of Venice](https://www.unive.it).

---

## Structure of this notebook

This notebook covers the following points
* The idea
* Tuning:
    * Automatic: GridSearchCV Hyperparameters tuning for XGBoost.
    * Manual
* Model validation
* Results
* Analysis of worst and best predictions.

---

### Before running this notebook

To avoid issues, before running the following notebook it is best to
* Clean previous cell outputs
* Restart the kernel

---

## The idea

As we know, different predictors have different flaws and strengths. This means we can train multiple models in order to exploit what they learnt and obtain a more accurate result.

As we are using a boosting method, we expect to find some level of overfitting when testing it on the dataset where the outliers and most noise were removed. 

Plus, as the dataset shows very few instances, it migth be better to use this kind of model on a larger dataset.

However, there should be some level of improvement given the boosting algorithm will try to lower the bias.

---

### Environment, Globals and Imports

In [1]:
!pip install mlxtend
!pip install xgboost

In [7]:
# Interactive
%matplotlib notebook
# Static
# %matplotlib inline

# Environment for this notebook
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import warnings
import sklearn 
import IPython
import xgboost
from scipy import stats
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold

# Set the style for the plots
sns.set()
plt.style.use('ggplot')
sns.set_style("darkgrid")
# Ignore warnings
warnings.filterwarnings('ignore') 

# Working folder
WORKING_DIR = os.getcwd()
# Resources folder
RESOURCES_DIR = os.path.join(os.getcwd(), 'resources')
# Name of file
IN_LABEL = 'ames_housing_out_21.csv'
IN_LABEL2 = 'ames_housing_out_22.csv'
ORIG_LABEL = 'ames_housing_out_22_orig.csv'

In [3]:
# Utils Module

def sort_alphabetically(dataset, last_label = None):
    """
    Sorts the dataset alphabetically 

    :param dataset: a pd.DataFrame
    :param last_label: a str containing an existing column label in the dataset
    :returns: pd.DataFrame
    """
    # Sort
    dataset = dataset.reindex(sorted(dataset.columns), axis=1)
    # Move target column to last index
    if last_label is not None:
        col = dataset.pop(last_label)
        dataset.insert(dataset.shape[1], last_label, col)
    return dataset

In [4]:
from sklearn.model_selection import train_test_split

# Module for train test split

def get_X_y(dataset, label, ignore=None):
    """
    Returns X and y and ignores labels in ignore
    :param dataset: a pd.DataFrame
    :param label: a str containing an existing target column label in the dataset
    :param ignore: a list of str containing an existing column label in the dataset to ignore
    :returns: tuple of pd.DataFrame
    """
    if ignore is not None:
        # Drop the labels 
        all_columns = list(dataset.columns)
        # Include only columns that are existing 
        to_drop = [i for i in all_columns if i in ignore] +[label]
        return dataset.drop(columns=to_drop), dataset[[label]]
    return dataset.drop(columns=[label]), dataset[[label]]

def get_train_test(X, y, size = 0.2, state = 33):
    """
    Returns X_train_[size], X_test, y_train_[size], y_test
    :param X: a pd.DataFrame without the target column
    :param y: a pd.DataFrame with one column, the target
    :param size: a float representing the fraction for the test size
    :param state: an integer representing the random state for the test
    :returns: 4 pd.DataFrame usually called "X_train_[size], X_test, y_train_[size], y_test"
    """
    return train_test_split(X, y, test_size=size, random_state = state)

def get_train_val_test(X, y, size_t=0.2, size_v=0.25, state_v = 42):
    """
    Returns X_train, X_valid, X_test, y_train, y_valid, y_test
    :param X: a pd.DataFrame without the target column
    :param y: a pd.DataFrame with one column, the target
    :param size_t: a float representing the fraction for the test size
    :param size_v: a float representing the fraction for the validation
    :param state_v: an integer representing the random state for the validation
    :returns: 6 pd.DataFrame usually called X_train, X_valid, X_test, y_train, y_valid, y_test
    """
    X_train_s, X_test, y_train_s, y_test = get_train_test(X, y, size = size_t)
    X_train, X_valid, y_train, y_valid = get_train_test(X_train_s, y_train_s, size = size_v, state = state_v)
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [5]:
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, r2_score, max_error 
from mlxtend.evaluate import bias_variance_decomp

# Module for traininig and testing
def get_regression_metrics(y_test, y_pred):
    metrics = {
            "RMSE": mean_squared_error(y_true=y_test, y_pred=y_pred, squared=False),
            "MSE": mean_squared_error(y_true=y_test, y_pred=y_pred),
            "MSLE": mean_squared_log_error(y_true=y_test, y_pred=y_pred),
            "MAE": mean_absolute_error(y_true=y_test, y_pred=y_pred),
            "R2": r2_score(y_true=y_test, y_pred=y_pred),
            "MAX_Err": max_error(y_true=y_test, y_pred=y_pred)}
    return metrics


def get_bias_variance_decomp(dataset, model, label, split_size, ignore, 
                             num_rounds=50, random_state=230324945):
    # Get split
    X, y = get_X_y(dataset, label=label, ignore=None)
    X_train, X_test, y_train, y_test = get_train_test(X, y, size = split_size, 
                                                      state = random_state)
    # Only accepts np.arrays
    mse, bias, var = bias_variance_decomp(estimator=model, 
                                          X_train=X_train.values, 
                                          y_train=y_train.values, 
                                          X_test=X_test.values, 
                                          y_test=y_test.values, 
                                          loss='mse', num_rounds=num_rounds, 
                                          random_seed=random_state)
    print('Avg Expected RMSE: %.3f' % np.sqrt(mse))
    print('Avg Expected MSE: %.3f' % mse)
    print('Avg Bias: %.3f' % bias)
    print('Avg Variance: %.3f' % var)
    pass


def LOO_estimator_eval(dataset, target, estimator, params, ignore=None):
    """
    Function used to evaluate estimators, based on Leave One Out process. It adds a 
    column 'Predicted' to the given dataset, and returns the metrics used to evaluate the 
    performances
    
    :param dataset: a pd.DataFrame with the target column
    :param target: a str representing the target
    :param estimator: instance of some estimator (i.e. XGBoostRegressor())
    :param params: a dictionary containing the parameters for the estimator
    :param ignor: a list of strings representing the feature to ignore
    :returns: the pd.DataFrame
    """
    # Splitter
    splitter = LeaveOneOut()
    
    # Add predicted
    dataset['Predicted'] = 0.0
    
    # Ignore
    if ignore is not None:
        ignore = ignore + ['Predicted']
    else:
        ignore = ['Predicted']
    
    # Split X, y
    X, y = get_X_y(dataset, label=target, ignore=ignore)
    
    # For each fold and tuple train, test indices
    for i, (train_index, test_index) in enumerate(splitter.split(X)):
        # Re-Assign
        model = estimator
        
        # Base model initialized with some parameters
        if params is not None: 
            print(params)
            model.set_params(params)
        
        # Get train part
        train = dataset.loc[train_index.tolist()]
        X_train, y_train = get_X_y(train, label=target, ignore=ignore)
       
        # Train 
        model.fit(X_train, y_train)

        # Get test part 
        test = dataset.loc[test_index.tolist()]
        X_test, y_test = get_X_y(test, label=target, ignore=ignore)
        
        # Add predict to dataset
        y_pred = model.predict(X_test)
        dataset.loc[test_index.tolist()[0]]['Predicted'] = y_pred[0]
    return get_regression_metrics(dataset[[target]], dataset[['Predicted']])


def GridSearch_CV_Tuning(dataset, target, estimator, params, ignore=None, n_repeats=4, n_splits=4, 
                random_state=33412):
    """
    Function used to evaluate estimators, based on GridSearchCV process. It evaluates the
    performances through a Repeated K Fold, and returns the results
    
    :param dataset: a pd.DataFrame with the target column
    :param target: a str representing the target
    :param estimator: instance of some estimator (i.e. XGBoostRegressor())
    :param params: a dictionary containing the parameters for the estimator
    :param ignore: a list of strings representing the feature to ignore
    :param n_repeats: a integer
    :param n_splits: a integer
    :returns: the pd.DataFrame containing the results
    """
    # Ignore
    if ignore is not None:
        ignore = ignore + ['Predicted']
    else:
        ignore = ['Predicted']
        
    # RepeatedKFold splitter
    splitter = RepeatedKFold(n_repeats=n_repeats, n_splits=n_splits, random_state=random_state)
    
    # GridSearchCV
    clf = GridSearchCV(estimator=estimator, cv=splitter,
                       param_grid=params, return_train_score = True,
                       scoring =['neg_mean_squared_error', 'neg_root_mean_squared_error', 'r2'],
                       refit=False, n_jobs=-1, verbose=3)
    # X, y
    X, y = get_X_y(dataset, label=target, ignore=(ignore + ['Predicted']))
    # Train, Test split
    X_train, X_test, y_train, y_test = get_train_test(X, y)
    # Fit
    clf.fit(X_train, y_train)
    
    return pd.DataFrame(clf.cv_results_)



## Dataset Overview

The dataset we are going to consider are the following ones:
* The modified dataset, in two different subset versions
* The original dataset

In [9]:
df = pd.read_csv(os.path.join(RESOURCES_DIR, IN_LABEL))
df2 = pd.read_csv(os.path.join(RESOURCES_DIR, IN_LABEL2))
df_orig = pd.read_csv(os.path.join(RESOURCES_DIR, ORIG_LABEL))

df.drop(columns=['Unnamed: 0', 'Latitude', 'Longitude'], inplace=True)
df_orig.drop(columns=['Unnamed: 0', 'Latitude', 'Longitude'], inplace=True)

df = sort_alphabetically(df, 'Sale_Price')
df2 = sort_alphabetically(df2, 'Sale_Price')
df_orig = sort_alphabetically(df_orig, 'Sale_Price')

---

## Hyperparameters Tuning

First of all, let us try to use a Grid Search CV to find the best parameters.

### Automatic Parameters Tuning: Randomized Grid Search

By defining the repetitions, the splits and the parameters, we repeatedly train and test the models. From each one of the model, we obtain three scores which we can use to check the best a parameters.

But first we can choose a parsimonious range of hyperparameters to test

In [13]:
xgb_params = {
    'n_estimators': [5, 7, 9, 11],
    'max_depth': [5, 7, 9, 11],
    'max_leaves': [6, 8, 10, 12],
    'learning_rate': [0.5, 0.75, 1],
    'booster' : ['gbtree'],
    'importance_type': ['weight', 'gain'],
}

In [14]:
results = GridSearch_CV_Tuning(dataset=df, target='Sale_Price', estimator=XGBRegressor(), params=xgb_params)

Fitting 16 folds for each of 384 candidates, totalling 6144 fits


From this dataframe we want to obtain the 10 best models for each metric we used. 

In [25]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_importance_type,param_learning_rate,param_max_depth,param_max_leaves,param_n_estimators,...,split8_train_r2,split9_train_r2,split10_train_r2,split11_train_r2,split12_train_r2,split13_train_r2,split14_train_r2,split15_train_r2,mean_train_r2,std_train_r2
0,0.203375,0.040059,0.006625,0.001053,gbtree,weight,0.5,5,6,5,...,0.939692,0.939879,0.937185,0.938392,0.938539,0.939914,0.938959,0.941348,0.939427,0.001812
1,0.028000,0.004924,0.006000,0.001500,gbtree,weight,0.5,5,6,7,...,0.960308,0.958681,0.958085,0.959365,0.957841,0.959687,0.961159,0.959330,0.959426,0.001174
2,0.041125,0.003919,0.007062,0.000428,gbtree,weight,0.5,5,6,9,...,0.967721,0.965205,0.965395,0.965554,0.966298,0.965548,0.967224,0.965820,0.966173,0.001291
3,0.049375,0.002369,0.006813,0.000527,gbtree,weight,0.5,5,6,11,...,0.971625,0.969861,0.968147,0.969142,0.970206,0.970058,0.970820,0.970608,0.970135,0.001262
4,0.026750,0.001090,0.007375,0.001053,gbtree,weight,0.5,5,8,5,...,0.939692,0.939879,0.937185,0.938392,0.938539,0.939914,0.938959,0.941348,0.939427,0.001812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0.112750,0.003648,0.006707,0.000459,gbtree,gain,1,11,10,11,...,0.999969,0.999801,0.999885,0.999830,0.999920,0.999821,0.999897,0.999960,0.999909,0.000050
380,0.052499,0.001732,0.006938,0.000555,gbtree,gain,1,11,12,5,...,0.996973,0.993098,0.994866,0.993211,0.995452,0.994067,0.994033,0.995728,0.995468,0.001226
381,0.071563,0.002669,0.007125,0.001654,gbtree,gain,1,11,12,7,...,0.999463,0.997293,0.998219,0.997493,0.998828,0.998224,0.998711,0.999527,0.998584,0.000623
382,0.091937,0.001952,0.006445,0.000490,gbtree,gain,1,11,12,9,...,0.999869,0.999187,0.999434,0.999270,0.999681,0.999340,0.999657,0.999906,0.999633,0.000210


In [16]:
best_r2 = list(results[['rank_test_r2','mean_train_r2', 'mean_test_r2']][results['rank_test_r2']==1].index)
best_mse = list(results[['rank_test_neg_mean_squared_error','mean_train_neg_mean_squared_error', 'mean_test_neg_mean_squared_error']][results['rank_test_neg_mean_squared_error']==1].index)
best_rmse = list(results[['rank_test_neg_root_mean_squared_error','mean_train_neg_root_mean_squared_error', 'mean_test_neg_root_mean_squared_error']][results['rank_test_neg_root_mean_squared_error']==1].index)

best = list(set(best_r2) | set(best_mse) | set(best_rmse))

In [17]:
best_df = results[['mean_fit_time', 'mean_test_neg_mean_squared_error', 'mean_test_neg_root_mean_squared_error', 'mean_test_r2', 'params',]].loc[best].sort_values(by=['mean_fit_time'])

In [18]:
best_df

,mean_fit_time,mean_test_neg_mean_squared_error,mean_test_neg_root_mean_squared_error,mean_test_r2,params
199,0.047000,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'gain..."
195,0.047687,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'gain..."
203,0.047875,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'gain..."
7,0.047999,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'weig..."
15,0.048125,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'weig..."
11,0.048312,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'weig..."
207,0.048375,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'gain..."
3,0.049375,-5.608038e+08,-23650.231297,0.89926,"{'booster': 'gbtree', 'importance_type': 'weig..."


In [19]:
pd.DataFrame(list(best_df.params))

,booster,importance_type,learning_rate,max_depth,max_leaves,n_estimators
0,gbtree,gain,0.5,5,8,11
1,gbtree,gain,0.5,5,6,11
2,gbtree,gain,0.5,5,10,11
3,gbtree,weight,0.5,5,8,11
4,gbtree,weight,0.5,5,12,11
5,gbtree,weight,0.5,5,10,11
6,gbtree,gain,0.5,5,12,11
7,gbtree,weight,0.5,5,6,11


Just to make sure this is the right way I want to re iter this on the original dataset

In [20]:
results_orig = GridSearch_CV_Tuning(dataset=df_orig, target='Sale_Price', estimator=XGBRegressor(), params=xgb_params)

Fitting 16 folds for each of 384 candidates, totalling 6144 fits


In [21]:
results_orig

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_importance_type,param_learning_rate,param_max_depth,param_max_leaves,param_n_estimators,...,split8_train_r2,split9_train_r2,split10_train_r2,split11_train_r2,split12_train_r2,split13_train_r2,split14_train_r2,split15_train_r2,mean_train_r2,std_train_r2
0,0.031613,0.000690,0.006688,0.000682,gbtree,weight,0.5,5,6,5,...,0.935909,0.938823,0.938464,0.938249,0.938388,0.938563,0.939364,0.935829,0.938349,0.002245
1,0.041187,0.003321,0.007125,0.000857,gbtree,weight,0.5,5,6,7,...,0.957856,0.958390,0.958449,0.958015,0.959361,0.957373,0.959001,0.956316,0.958192,0.001818
2,0.050437,0.003742,0.007125,0.002088,gbtree,weight,0.5,5,6,9,...,0.964789,0.964925,0.964157,0.964777,0.965492,0.962529,0.966543,0.962486,0.964714,0.001865
3,0.058125,0.002395,0.006563,0.000864,gbtree,weight,0.5,5,6,11,...,0.969684,0.968406,0.969254,0.968750,0.969474,0.965576,0.971125,0.967228,0.968847,0.001954
4,0.031936,0.001088,0.006438,0.000788,gbtree,weight,0.5,5,8,5,...,0.935909,0.938823,0.938464,0.938249,0.938388,0.938563,0.939364,0.935829,0.938349,0.002245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0.135250,0.003579,0.006625,0.000484,gbtree,gain,1,11,10,11,...,0.999856,0.999850,0.999924,0.999879,0.999720,0.999894,0.999860,0.999878,0.999886,0.000052
380,0.061438,0.001322,0.006500,0.000612,gbtree,gain,1,11,12,5,...,0.991559,0.991901,0.993149,0.993012,0.994604,0.994297,0.994597,0.993391,0.993604,0.001169
381,0.087124,0.002803,0.006815,0.000632,gbtree,gain,1,11,12,7,...,0.997510,0.997114,0.998506,0.998340,0.997651,0.998103,0.998580,0.997845,0.998100,0.000446
382,0.111750,0.003717,0.006750,0.000661,gbtree,gain,1,11,12,9,...,0.999495,0.999301,0.999580,0.999632,0.999307,0.999561,0.999461,0.999338,0.999560,0.000146


In [22]:
best_r22 = list(results_orig[['rank_test_r2','mean_train_r2', 'mean_test_r2']][results_orig['rank_test_r2']==1].index)
best_mse2 = list(results_orig[['rank_test_neg_mean_squared_error','mean_train_neg_mean_squared_error', 'mean_test_neg_mean_squared_error']][results_orig['rank_test_neg_mean_squared_error']==1].index)
best_rmse2 = list(results_orig[['rank_test_neg_root_mean_squared_error','mean_train_neg_root_mean_squared_error', 'mean_test_neg_root_mean_squared_error']][results_orig['rank_test_neg_root_mean_squared_error']==1].index)

best2 = list(set(best_r22) | set(best_mse2) | set(best_rmse2))

best_df2 = results_orig[['mean_fit_time', 'mean_test_neg_mean_squared_error', 'mean_test_neg_root_mean_squared_error', 'mean_test_r2', 'params',]].loc[best2].sort_values(by=['mean_fit_time'])

In [23]:
best_df2 

,mean_fit_time,mean_test_neg_mean_squared_error,mean_test_neg_root_mean_squared_error,mean_test_r2,params
3,0.058125,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."
199,0.058188,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
195,0.058438,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
203,0.058937,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
7,0.059875,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."
11,0.060063,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."
207,0.060687,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
15,0.061938,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."


In [24]:
pd.DataFrame(list(best_df2.params))

,booster,importance_type,learning_rate,max_depth,max_leaves,n_estimators
0,gbtree,weight,0.5,5,6,11
1,gbtree,gain,0.5,5,8,11
2,gbtree,gain,0.5,5,6,11
3,gbtree,gain,0.5,5,10,11
4,gbtree,weight,0.5,5,8,11
5,gbtree,weight,0.5,5,10,11
6,gbtree,gain,0.5,5,12,11
7,gbtree,weight,0.5,5,12,11


In [26]:
results2 = GridSearch_CV_Tuning(dataset=df2, target='Sale_Price', estimator=XGBRegressor(), params=xgb_params)

Fitting 16 folds for each of 384 candidates, totalling 6144 fits


In [27]:
results2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_importance_type,param_learning_rate,param_max_depth,param_max_leaves,param_n_estimators,...,split8_train_r2,split9_train_r2,split10_train_r2,split11_train_r2,split12_train_r2,split13_train_r2,split14_train_r2,split15_train_r2,mean_train_r2,std_train_r2
0,0.032362,0.001469,0.006250,0.000433,gbtree,weight,0.5,5,6,5,...,0.944568,0.941767,0.943696,0.941051,0.941399,0.945594,0.944625,0.942266,0.943836,0.001773
1,0.041687,0.001488,0.006313,0.001211,gbtree,weight,0.5,5,6,7,...,0.964084,0.961504,0.964316,0.961951,0.962806,0.965201,0.963793,0.961944,0.963492,0.001656
2,0.052936,0.002512,0.006313,0.000463,gbtree,weight,0.5,5,6,9,...,0.970178,0.968268,0.973004,0.967771,0.969868,0.972482,0.969589,0.967652,0.970085,0.001639
3,0.063751,0.003113,0.007062,0.000899,gbtree,weight,0.5,5,6,11,...,0.972959,0.972534,0.977260,0.971381,0.973782,0.976215,0.974092,0.972656,0.974491,0.001654
4,0.034748,0.002221,0.006626,0.000484,gbtree,weight,0.5,5,8,5,...,0.944568,0.941767,0.943696,0.941051,0.941399,0.945594,0.944625,0.942266,0.943836,0.001773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0.143123,0.003119,0.007068,0.000961,gbtree,gain,1,11,10,11,...,0.999868,0.999923,0.999906,0.999869,0.999894,0.999907,0.999888,0.999849,0.999898,0.000037
380,0.066813,0.002215,0.006875,0.000484,gbtree,gain,1,11,12,5,...,0.994334,0.995166,0.996083,0.995620,0.996101,0.996302,0.994962,0.995141,0.995194,0.001346
381,0.090745,0.001554,0.006562,0.000609,gbtree,gain,1,11,12,7,...,0.998712,0.998670,0.998691,0.998546,0.999242,0.998843,0.998057,0.998055,0.998611,0.000460
382,0.116499,0.002500,0.007000,0.000354,gbtree,gain,1,11,12,9,...,0.999498,0.999467,0.999617,0.999442,0.999754,0.999670,0.999379,0.999605,0.999614,0.000124


In [28]:
best_r3 = list(results2[['rank_test_r2','mean_train_r2', 'mean_test_r2']][results2['rank_test_r2']==1].index)
best_mse3 = list(results2[['rank_test_neg_mean_squared_error','mean_train_neg_mean_squared_error', 'mean_test_neg_mean_squared_error']][results2['rank_test_neg_mean_squared_error']==1].index)
best_rmse3 = list(results2[['rank_test_neg_root_mean_squared_error','mean_train_neg_root_mean_squared_error', 'mean_test_neg_root_mean_squared_error']][results2['rank_test_neg_root_mean_squared_error']==1].index)

best3 = list(set(best_r3) | set(best_mse3) | set(best_rmse3))

best_df3 = results_orig[['mean_fit_time', 'mean_test_neg_mean_squared_error', 'mean_test_neg_root_mean_squared_error', 'mean_test_r2', 'params',]].loc[best3].sort_values(by=['mean_fit_time'])

In [29]:
best_df3

,mean_fit_time,mean_test_neg_mean_squared_error,mean_test_neg_root_mean_squared_error,mean_test_r2,params
3,0.058125,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."
199,0.058188,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
195,0.058438,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
203,0.058937,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
7,0.059875,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."
11,0.060063,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."
207,0.060687,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'gain..."
15,0.061938,-8.385927e+08,-28775.492507,0.863008,"{'booster': 'gbtree', 'importance_type': 'weig..."


In [30]:
pd.DataFrame(list(best_df3.params))

,booster,importance_type,learning_rate,max_depth,max_leaves,n_estimators
0,gbtree,weight,0.5,5,6,11
1,gbtree,gain,0.5,5,8,11
2,gbtree,gain,0.5,5,6,11
3,gbtree,gain,0.5,5,10,11
4,gbtree,weight,0.5,5,8,11
5,gbtree,weight,0.5,5,10,11
6,gbtree,gain,0.5,5,12,11
7,gbtree,weight,0.5,5,12,11


---

## XGBoost Evaluation - Leave One Out Metrics Computing

Now that we have some good knowledge about hyperparameters for the estimator, we can closely analyze how accurate the model is.

To get the most accurate results on the test, we are going to use the Leave One Out Approach for the instances. The model is going to be evaluated multiple times and the dataset will be changed in place. 
This is going to allow us to find the best and worst predictions.

In [31]:
test_df1 = df.copy()
LOO_estimator_eval(dataset=test_df1, target='Sale_Price', 
                   estimator=XGBRegressor(
                       booster='gbtree', importance_type='weight', learning_rate = 0.5,
                       max_depth=5, max_leaves=6, n_estimators = 11), 
                   params=None, ignore=None)

{'RMSE': 23420.254868662523,
 'MSE': 548508338.1131107,
 'MSLE': 0.015305551513536499,
 'MAE': 16033.898817253621,
 'R2': 0.9059901574555764,
 'MAX_Err': 171266.25}

In [32]:
test_df2 = df.copy()
LOO_estimator_eval(dataset=test_df2, target='Sale_Price', 
                   estimator=XGBRegressor(
                       booster='gbtree', importance_type='gain', learning_rate = 0.5,
                       max_depth=5, max_leaves=8, n_estimators = 11), 
                   params=None, ignore=None)

{'RMSE': 23420.254868662523,
 'MSE': 548508338.1131107,
 'MSLE': 0.015305551513536499,
 'MAE': 16033.898817253621,
 'R2': 0.9059901574555764,
 'MAX_Err': 171266.25}

We can see that for the first dataset, the most parsimonious model is also the best one!

In [33]:
test_df3 = df2.copy()
LOO_estimator_eval(dataset=test_df3, target='Sale_Price', 
                   estimator=XGBRegressor(
                       booster='gbtree', importance_type='weight', learning_rate = 0.5,
                       max_depth=5, max_leaves=6, n_estimators = 11), 
                   params=None, ignore=None)

{'RMSE': 198156.87459070655,
 'MSE': 39266146947.557,
 'MSLE': 145.12771146453164,
 'MAE': 182843.00038124286,
 'R2': -5.7298964033338144,
 'MAX_Err': 615000.0}

In [34]:
test_df4 = df2.copy()
LOO_estimator_eval(dataset=test_df4, target='Sale_Price', 
                   estimator=XGBRegressor(
                       booster='gbtree', importance_type='gain', learning_rate = 0.5,
                       max_depth=5, max_leaves=8, n_estimators = 11), 
                   params=None, ignore=None)

{'RMSE': 198156.87459070655,
 'MSE': 39266146947.557,
 'MSLE': 145.12771146453164,
 'MAE': 182843.00038124286,
 'R2': -5.7298964033338144,
 'MAX_Err': 615000.0}

Curiosly enough, the RMSE is much smaller here, but the RSquared Value is looking very bad.

In [35]:
test_df5 = df_orig.copy()
LOO_estimator_eval(dataset=test_df5, target='Sale_Price', 
                   estimator=XGBRegressor(
                       booster='gbtree', importance_type='weight', learning_rate = 0.5,
                       max_depth=5, max_leaves=8, n_estimators = 11), 
                   params=None, ignore=None)

{'RMSE': 28143.959008642858,
 'MSE': 792082428.6801695,
 'MSLE': 0.020786093949075216,
 'MAE': 17273.812734641637,
 'R2': 0.8758434180265611,
 'MAX_Err': 371575.515625}

In [36]:
test_df6 = df_orig.copy()
LOO_estimator_eval(dataset=test_df6, target='Sale_Price', 
                   estimator=XGBRegressor(
                       booster='gbtree', importance_type='gain', learning_rate = 0.5,
                       max_depth=5, max_leaves=8, n_estimators = 11), 
                   params=None, ignore=None)

{'RMSE': 28143.959008642858,
 'MSE': 792082428.6801695,
 'MSLE': 0.020786093949075216,
 'MAE': 17273.812734641637,
 'R2': 0.8758434180265611,
 'MAX_Err': 371575.515625}

## Worst Predictions and Best Predictions

To consider the best and worst predictions, I decided to consider the models with the lowest error.

In [38]:
# Modified First Subset
test_df1['Prediction_Error'] = np.abs(test_df1['Sale_Price']-test_df1['Predicted'])
# Original Second Subset
test_df6['Prediction_Error'] = np.abs(test_df6['Sale_Price']-test_df6['Predicted'])

### Most wrong on df

In [39]:
test_df1.sort_values(by=['Prediction_Error', 'Sale_Price'], ascending=False).head(10)

,BC_Bsmt_Unf_SF,Bedroom_AbvGr,Bsmt_Full_Bath,Bsmt_Unf_SF,Central_Air,External_Eval,Fireplace_Gr_Area_Ratio,Fireplace_Qu,Garage_Area,Garage_Cars,...,garage_type_1,hs_style_1,neighborhoods_1,neighborhoods_2,neighborhoods_3,neighborhoods_4,sale_cond_1,Sale_Price,Predicted,Prediction_Error
1601,4.229794,2.0,0.0,1795.0,0.0,98.000000,1795.0,4.0,895.0,3.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,147000.0,318266.250000,171266.250000
1191,3.545612,4.0,0.0,425.0,1.0,118.400000,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,122000.0,285493.593750,163493.593750
2218,2.482185,4.0,1.0,48.0,0.0,2681.800000,3500.0,3.0,959.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,584500.0,432576.000000,151924.000000
957,3.341847,1.0,2.0,278.0,0.0,1563.299900,1235.0,4.0,789.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,615000.0,482256.125000,132743.875000
1524,4.315548,3.0,0.0,2153.0,0.0,1344.599900,2643.0,4.0,694.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,380000.0,508107.656250,128107.656250
389,4.213478,4.0,0.0,1734.0,0.0,1117.799900,2822.0,4.0,1020.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,582933.0,456406.156250,126526.843750
1469,3.597804,2.0,1.0,474.0,0.0,280.000000,1419.0,4.0,567.0,2.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,392000.0,268796.093750,123203.906250
388,3.441461,2.0,2.0,342.0,0.0,1106.999900,1337.0,4.0,762.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,610000.0,487121.250000,122878.750000
2037,3.421390,1.0,1.0,328.0,0.0,0.000000,1502.0,3.0,286.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,330000.0,207637.312500,122362.687500
40,3.016554,2.0,1.0,142.0,0.0,180.900000,1182.0,4.0,820.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,611657.0,494127.000000,117530.000000


In [42]:
test_df1.sort_values(by=['Prediction_Error', 'Sale_Price'], ascending=False).head(50).describe()

,BC_Bsmt_Unf_SF,Bedroom_AbvGr,Bsmt_Full_Bath,Bsmt_Unf_SF,Central_Air,External_Eval,Fireplace_Gr_Area_Ratio,Fireplace_Qu,Garage_Area,Garage_Cars,...,garage_type_1,hs_style_1,neighborhoods_1,neighborhoods_2,neighborhoods_3,neighborhoods_4,sale_cond_1,Sale_Price,Predicted,Prediction_Error
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.00000,50.00000,50.000000,50.000000,50.000000,50.000000
mean,3.368790,3.120000,0.540000,764.200000,0.040000,633.975980,1620.180000,3.080000,613.640000,2.280000,...,0.880000,0.920000,0.160000,0.020000,0.14000,0.30000,0.620000,323964.260000,299337.077500,94251.421875
std,1.109907,1.099907,0.613122,666.234591,0.197949,558.912872,991.684445,1.588736,238.886512,0.809132,...,0.328261,0.274048,0.370328,0.141421,0.35051,0.46291,0.490314,132042.402367,104149.206779,25378.589595
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,97500.000000,133930.328125,68075.468750
25%,3.101604,2.000000,0.000000,169.500000,0.000000,185.675000,1005.125000,3.000000,495.000000,2.000000,...,1.000000,1.000000,0.000000,0.000000,0.00000,0.00000,0.000000,236000.000000,214416.976562,74806.179688
50%,3.636660,3.000000,0.000000,514.500000,0.000000,545.000000,1711.500000,4.000000,576.000000,2.000000,...,1.000000,1.000000,0.000000,0.000000,0.00000,0.00000,1.000000,321500.000000,286101.750000,85241.140625
75%,4.096378,4.000000,1.000000,1354.750000,0.000000,920.175000,2507.250000,4.000000,784.250000,3.000000,...,1.000000,1.000000,0.000000,0.000000,0.00000,1.00000,1.000000,382977.500000,368434.148438,105025.480469
max,4.315548,6.000000,2.000000,2153.000000,1.000000,2681.800000,3500.000000,5.000000,1052.000000,3.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000,615000.000000,508107.656250,171266.250000


### Best on df

In [40]:
test_df1.sort_values(by=['Prediction_Error', 'Sale_Price']).head(10)

,BC_Bsmt_Unf_SF,Bedroom_AbvGr,Bsmt_Full_Bath,Bsmt_Unf_SF,Central_Air,External_Eval,Fireplace_Gr_Area_Ratio,Fireplace_Qu,Garage_Area,Garage_Cars,...,garage_type_1,hs_style_1,neighborhoods_1,neighborhoods_2,neighborhoods_3,neighborhoods_4,sale_cond_1,Sale_Price,Predicted,Prediction_Error
2310,3.562210,2.0,0.0,440.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,85500.0,85484.546875,15.453125
463,3.973080,3.0,0.0,1043.0,0.0,694.000000,1432.0,4.0,492.0,2.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,245000.0,245021.968750,21.968750
2462,3.979399,3.0,0.0,1057.0,0.0,0.000000,0.0,0.0,288.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,137000.0,137022.953125,22.953125
1270,3.373666,3.0,0.0,297.0,0.0,472.000000,0.0,0.0,280.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,134500.0,134534.062500,34.062500
1626,3.472692,3.0,1.0,365.0,0.0,0.000000,0.0,0.0,576.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,149000.0,148950.078125,49.921875
654,3.783721,4.0,0.0,700.0,0.0,388.699980,1742.0,4.0,424.0,2.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,168000.0,168052.265625,52.265625
1880,3.529576,3.0,0.0,411.0,0.0,0.000000,0.0,0.0,399.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,118000.0,118053.171875,53.171875
82,3.553431,3.0,0.0,432.0,0.0,0.000000,0.0,0.0,280.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,122000.0,122053.960938,53.960938
2327,3.857784,3.0,0.0,818.0,0.0,58.499996,0.0,0.0,292.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,145250.0,145194.125000,55.875000
965,4.094168,2.0,0.0,1347.0,0.0,66.000000,1367.0,4.0,484.0,2.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,192000.0,191943.046875,56.953125


In [41]:
test_df1.sort_values(by=['Prediction_Error', 'Sale_Price']).head(50).describe()

,BC_Bsmt_Unf_SF,Bedroom_AbvGr,Bsmt_Full_Bath,Bsmt_Unf_SF,Central_Air,External_Eval,Fireplace_Gr_Area_Ratio,Fireplace_Qu,Garage_Area,Garage_Cars,...,garage_type_1,hs_style_1,neighborhoods_1,neighborhoods_2,neighborhoods_3,neighborhoods_4,sale_cond_1,Sale_Price,Predicted,Prediction_Error
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.00000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,3.326493,2.780000,0.340000,578.800000,0.060000,187.477998,603.080000,1.500000,399.160000,1.520000,...,0.580000,0.760000,0.70000,0.020000,0.14000,0.040000,0.980000,153716.000000,153690.402344,168.007031
std,1.054502,0.887326,0.478518,409.131933,0.239898,210.001301,761.514632,1.821078,208.550817,0.735125,...,0.498569,0.431419,0.46291,0.141421,0.35051,0.197949,0.141421,52616.944039,52619.138127,113.449095
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,72000.000000,71654.640625,15.453125
25%,3.374874,2.000000,0.000000,297.750000,0.000000,1.300000,0.000000,0.000000,280.000000,1.000000,...,0.000000,1.000000,0.00000,0.000000,0.00000,0.000000,1.000000,118375.000000,118384.095703,66.750000
50%,3.608643,3.000000,0.000000,485.000000,0.000000,89.199997,0.000000,0.000000,399.500000,2.000000,...,1.000000,1.000000,1.00000,0.000000,0.00000,0.000000,1.000000,138250.000000,138048.171875,124.121094
75%,3.856617,3.000000,1.000000,816.000000,0.000000,352.274995,1373.750000,3.750000,490.000000,2.000000,...,1.000000,1.000000,1.00000,0.000000,0.00000,0.000000,1.000000,181175.000000,181150.703125,277.234375
max,4.184280,6.000000,1.000000,1630.000000,1.000000,732.000000,2287.000000,4.000000,907.000000,3.000000,...,1.000000,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,332200.000000,332524.781250,395.593750


### Most wrong on df_orig

In [43]:
test_df6.sort_values(by=['Prediction_Error', 'Sale_Price'], ascending=False).head(10)

,Age,BC_Bsmt_Unf_SF,BC_External_SF,Baths,Bedroom_Liv_Area_Ratio,Bsmt_Eval,Bsmt_Unf_SF,Central_Air,Exter_Qual,Fireplace_Eval,...,bldg_type_3,garage_type_1,hs_style_1,neighborhoods_1,neighborhoods_3,neighborhoods_4,sale_cond_1,Sale_Price,Predicted,Prediction_Error
423,1.0,3.851456,20.762966,3.0,1201.0000,12840.1010,788.0,0.0,3.0,2402.0000,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,555000.0,183424.484375,371575.515625
2181,0.0,3.903065,31.517199,4.5,1558.6666,13022.7000,878.0,0.0,3.0,9352.0000,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,184750.0,544274.875000,359524.875000
2666,114.0,4.013484,32.752815,2.5,902.0000,3099.5999,1107.0,0.0,2.0,3124.6196,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,475000.0,165385.421875,309614.578125
1182,31.0,3.708188,20.123798,3.0,981.3333,3056.5498,584.0,0.0,2.0,5888.0000,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,150000.0,382997.406250,232997.406250
1637,1.0,4.176208,27.650692,3.5,584.5000,10507.0000,1559.0,0.0,3.0,2338.0000,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,591587.0,403816.093750,187770.906250
2737,71.0,4.128849,35.326355,3.5,734.4000,4773.5996,1411.0,0.0,1.0,3672.0000,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,415000.0,238224.296875,176775.703125
2570,88.0,3.017334,40.354954,3.5,778.0000,3535.9998,140.0,0.0,2.0,6958.6436,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,235000.0,409790.281250,174790.281250
1063,5.0,3.350989,30.710903,3.5,2470.0000,10520.2500,278.0,0.0,3.0,2470.0000,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,615000.0,441111.156250,173888.843750
1767,13.0,3.959812,26.965897,4.0,1079.0000,10142.6010,989.0,0.0,3.0,4825.4346,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,755000.0,586041.062500,168958.937500
1782,1.0,4.243074,9.692594,2.0,897.5000,7090.2500,1795.0,0.0,2.0,3590.0000,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,147000.0,310691.781250,163691.781250


In [44]:
test_df6.sort_values(by=['Prediction_Error', 'Sale_Price'], ascending=False).head(50).describe()

,Age,BC_Bsmt_Unf_SF,BC_External_SF,Baths,Bedroom_Liv_Area_Ratio,Bsmt_Eval,Bsmt_Unf_SF,Central_Air,Exter_Qual,Fireplace_Eval,...,bldg_type_3,garage_type_1,hs_style_1,neighborhoods_1,neighborhoods_3,neighborhoods_4,sale_cond_1,Sale_Price,Predicted,Prediction_Error
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,26.220000,3.588896,23.185593,3.120000,895.531331,6969.808950,841.620000,0.040000,2.060000,3555.082086,...,0.020000,0.820000,0.86000,0.260000,0.080000,0.260000,0.460000,347418.620000,334127.271875,133319.100000
std,34.896518,0.859469,10.053021,0.759699,428.043946,3867.733648,627.092542,0.197949,0.766918,1917.580983,...,0.141421,0.388088,0.35051,0.443087,0.274048,0.443087,0.503457,173751.304594,124464.890199,64426.147216
min,0.000000,0.000000,0.000000,2.000000,288.000000,1556.099900,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,82500.000000,131277.468750,78640.968750
25%,1.000000,3.362793,18.682363,2.500000,654.062475,4812.955950,285.000000,0.000000,1.250000,2385.500000,...,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,221500.000000,241259.015625,93618.312500
50%,6.000000,3.780512,23.468677,3.000000,768.500000,6365.600000,679.500000,0.000000,2.000000,3281.000000,...,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,315750.000000,316755.671875,114122.218750
75%,41.750000,4.128764,28.597953,3.500000,986.416675,8164.300000,1410.750000,0.000000,3.000000,4586.651950,...,0.000000,1.000000,1.00000,0.750000,0.000000,0.750000,1.000000,441446.750000,408296.734375,144971.804688
max,136.000000,4.329243,47.315372,4.500000,2470.000000,25356.500000,2153.000000,1.000000,3.000000,9352.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,755000.000000,611544.937500,371575.515625


### Best on df

In [47]:
test_df6.sort_values(by=['Prediction_Error', 'Sale_Price']).head(10)

,Age,Alley_Access,Baths,Bedroom_AbvGr,Bedroom_Liv_Area_Ratio,Bsmt,Bsmt_Eval,Bsmt_Unf_SF,Central_Air,Electrical_SBrkr,...,TotBath_LivArea_Ratio,TotRms_AbvGrd,Total_Bsmt_SF,Total_SF,Year_Sold,neighborhoods_1,neighborhoods_5,Sale_Price,Predicted,Prediction_Error
2464,1.0,0.0,2.5,3.0,517.66670,1.0,2324.7000,756.0,0.0,1.0,...,621.2000,6.0,756.0,2309.0,2006.0,0.0,0.0,186500.0,186499.171875,0.828125
803,16.0,0.0,3.5,3.0,591.66670,1.0,2275.4998,227.0,0.0,1.0,...,710.0000,7.0,740.0,2515.0,2009.0,1.0,0.0,213000.0,212992.625000,7.375000
2050,43.0,0.0,2.0,3.0,335.00000,1.0,2613.0000,348.0,0.0,1.0,...,1005.0000,5.0,1005.0,2010.0,2007.0,1.0,0.0,115400.0,115421.437500,21.437500
2018,78.0,0.0,1.0,2.0,427.00000,1.0,2163.2000,832.0,0.0,0.0,...,854.0000,5.0,832.0,1686.0,2007.0,1.0,0.0,132000.0,132021.625000,21.625000
327,37.0,0.0,2.5,2.0,472.50000,1.0,3472.8750,30.0,0.0,1.0,...,945.0000,5.0,945.0,1890.0,2010.0,0.0,0.0,119500.0,119532.078125,32.078125
1497,49.0,0.0,3.5,5.0,764.00000,0.0,0.0000,0.0,0.0,1.0,...,1091.4286,11.0,0.0,3820.0,2008.0,1.0,0.0,284700.0,284664.875000,35.125000
2057,36.0,0.0,2.0,2.0,434.00000,1.0,2668.7998,20.0,0.0,1.0,...,868.0000,6.0,768.0,1636.0,2007.0,1.0,0.0,119900.0,119847.093750,52.906250
2072,23.0,0.0,2.0,3.0,507.33334,1.0,3659.2498,0.0,0.0,1.0,...,761.0000,7.0,1190.0,2712.0,2007.0,0.0,0.0,182000.0,182053.062500,53.062500
2720,49.0,0.0,3.0,3.0,601.00000,1.0,2948.4000,284.0,0.0,1.0,...,901.5000,8.0,1134.0,2937.0,2006.0,1.0,0.0,155000.0,154946.109375,53.890625
1623,11.0,0.0,2.5,3.0,517.66670,1.0,2124.8250,277.0,0.0,1.0,...,621.2000,6.0,691.0,2244.0,2007.0,0.0,0.0,178750.0,178693.562500,56.437500


In [45]:
test_df6.sort_values(by=['Prediction_Error', 'Sale_Price']).head(50).describe()

,Age,BC_Bsmt_Unf_SF,BC_External_SF,Baths,Bedroom_Liv_Area_Ratio,Bsmt_Eval,Bsmt_Unf_SF,Central_Air,Exter_Qual,Fireplace_Eval,...,bldg_type_3,garage_type_1,hs_style_1,neighborhoods_1,neighborhoods_3,neighborhoods_4,sale_cond_1,Sale_Price,Predicted,Prediction_Error
count,50.00000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.0,50.00000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,32.02000,3.257831,13.188114,2.21000,528.880000,3461.347952,575.980000,0.0,1.48000,1341.020232,...,0.160000,0.720000,0.820000,0.520000,0.160000,0.200000,0.760000,176000.680000,175951.847969,162.118906
std,27.03021,1.072057,8.851206,0.70051,275.046371,1381.489824,542.450731,0.0,0.67733,1572.078932,...,0.370328,0.453557,0.388088,0.504672,0.370328,0.404061,0.431419,72205.008434,72187.339161,87.100631
min,0.00000,0.000000,0.000000,1.00000,245.000000,1478.400000,0.000000,0.0,1.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,86000.000000,85901.390625,4.562500
25%,4.00000,3.104215,7.308492,2.00000,385.500000,2484.950000,167.250000,0.0,1.00000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,132625.000000,132509.878906,100.000000
50%,31.00000,3.511199,15.247634,2.00000,489.291670,3079.200000,387.500000,0.0,1.00000,0.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,145750.000000,145707.359375,157.265625
75%,48.75000,3.883149,19.417033,2.50000,600.062500,4197.762200,844.500000,0.0,2.00000,2685.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,187375.000000,187250.054688,219.585938
max,93.00000,4.322152,28.365940,4.00000,2132.000000,8802.150000,2121.000000,0.0,3.00000,4742.700000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,415298.000000,415195.656250,334.218750


---

### Final Comment

The XGBoost Regressor tested present a lot of variance in their results, which is hard to account for.

The most probable cause are the number of estimators.